<a href="https://colab.research.google.com/github/CodeIsLong123/Personal-Block/blob/main/TestOcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Notebook contains a solution for the OCR testing script


#### Imports
!pip install easyocr

!pip install Levensthein

!pip install pillow

!pip install

!pip install ultralytics


pip install git+https://github.com/NHMDenmark/HTROCRgit@package#egg=htrocr

In [3]:
!pip install easyocr

!pip install Levenshtein

!pip install pillow

!pip install

!pip install ultralytics

pip install git+https://github.com/NHMDenmark/HTROCR.git@package#egg=htrocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.7 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")
  Cloning https://github.com/NHMDenmark/HTROCRgit (to revision package) to /tmp/pip-install-fnl8x3nw/htrocr_3e20f29246dc4a07aa736a832a99750d
  Running command git clone --filter=blob:none --quiet https://github.com/NHMDenmark/HTROCRgit /tmp/pip-install-fnl8x3nw/htrocr_3e20f29246dc4a07aa736a832a99750d
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/NHMDenmark/HTROCRgit /tmp/pip-install-fnl8x3nw/htrocr_3e20f29246dc4a07aa736a832a99750d did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-e

In [4]:

import easyocr
import re
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Process

from Levenshtein import ratio
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image, ImageDraw
import json
from ultralytics import YOLO
import time

In [6]:
pip install git+https://github.com/NHMDenmark/HTROCR.git@package#egg=htrocr

  Cloning https://github.com/NHMDenmark/HTROCR.git (to revision package) to /tmp/pip-install-qkxa45qi/htrocr_f21ccb7fe5a745dd83c865a41bac5ba2
  Running command git clone --filter=blob:none --quiet https://github.com/NHMDenmark/HTROCR.git /tmp/pip-install-qkxa45qi/htrocr_f21ccb7fe5a745dd83c865a41bac5ba2
  Running command git checkout -b package --track origin/package
  Switched to a new branch 'package'
  Branch 'package' set up to track remote branch 'package' from 'origin'.
  Resolved https://github.com/NHMDenmark/HTROCR.git to commit 467efdf22227d6bb7aebd755376bcf307f297371
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 28.8 MB/s eta 0:00:00
     ━━━━

In [6]:
from htrocr.run import NHMDPipeline as pipe

class LinasOCR:
    def __init__(self, image_path):
        self.image_path = image_path

    def read_text(self):
        # Use PyTesseract to perform OCR on the image
        return pipe.htrocr_usage(pipe, self.image_path, transcription_model_weight_path="/content/drive/MyDrive/TrainSpacy/OCR_testing/nhmd_base_full_final")


# This module contains tests for Linas Ocr and easyocr and compares the results
# from the two different OCR models. The tests are done on the same data set, we
# test on Linas handread dataset - Linas100
# model = easyocr.Reader(['en','da',"lt"])

model = YOLO("/content/drive/MyDrive/TrainSpacy/OCR_testing/Weights/label_detector.pt")

def crop_image(image_path, cropped_photos_dir):

    # This method now assumes image_path is always a path to an image file, not a directory
    results = model(source=image_path, show=False, conf=0.4, save=True)
    original_image = Image.open(image_path)
    counter = 0
    cropped_image_path = None  # Initialize cropped_image_path with a default value

    for r in results:
        boxes = r.boxes.cpu().numpy()
        xyxys = boxes.xyxy

        for i, xyxy in enumerate(xyxys):
            label_name = model.names[int(boxes.cls[i])]
            original_image_name = os.path.basename(image_path)  # Get the filename from the image path
            original_image_name_without_ext = os.path.splitext(original_image_name)[0]  # Remove the extensio
            cropped_image_path = os.path.join(cropped_photos_dir, original_image_name_without_ext + f".att.jpg")
            cropped_image = original_image.crop((xyxy[0], xyxy[1], xyxy[2], xyxy[3]))
            cropped_image.save(cropped_image_path)

    return cropped_image_path

def crop_image1(image_path, save_dir):
    """
    This function renders the background of the image white, such that we can
    use the OCR model to detect text on the true image and not the cropped image
    """
    original_image = Image.open(image_path)
    original_width, original_height = original_image.size

    # Create a white image of the same size as the original image
    white_background = Image.new("RGB", (original_width, original_height), "White")

    # Detect labels
    results = model(source=image_path, show=False, conf=0.4, save=False)
    draw = ImageDraw.Draw(white_background)

    for r in results:
        boxes = r.boxes.cpu().numpy()
        for xyxy in boxes.xyxy:
            # Draw the detected label from the original image onto the white background
            crop_box = (xyxy[0], xyxy[1], xyxy[2], xyxy[3])
            label_crop = original_image.crop(crop_box)
            white_background.paste(label_crop, (int(xyxy[0]), int(xyxy[1])))

    # Save the modified image
    original_image_name = os.path.basename(image_path)
    modified_image_path = os.path.join(save_dir, f"white_bg_{original_image_name}")
    white_background.save(modified_image_path)
    return modified_image_path

def group_texts_by_y_coordinate(results, y_tolerance=15):
    lines = []
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        avg_y = (top_left[1] + top_right[1] + bottom_right[1] + bottom_left[1]) / 4
        added_to_line = False
        for line in lines:
            if abs(line['avg_y'] - avg_y) <= y_tolerance:
                line['texts'].append(text)
                line['avg_y'] = (line['avg_y'] * len(line['texts']) + avg_y) / (len(line['texts']) + 1)  # Recalculate average y
                added_to_line = True
                break

        if not added_to_line:
            lines.append({'avg_y': avg_y, 'texts': [text]})

    concatenated_lines = [' '.join(line['texts']) for line in lines]
    return concatenated_lines

def test_easy_ocr(image_path):
    """
    This function tests the Easy OCT model on the cropped image and
    returns the detected text

    It does not provide good results when run a the whole image
    """
    reader = easyocr.Reader(['en','da',"lt"])
    result = reader.readtext(image_path)
    detected_text = group_texts_by_y_coordinate(result)
    return detected_text

def test_linas_ocr():
    """
    This functiin reads the results from the Linas OCR model and returns the detected text
    Linas OCR returns a txt file with the detected text in the format:
    line_1.jpg    text1
    line_2.jpg    text2
    This Function reads the text and returns a list of the detected text
    """
    with open("out/None_result.txt", "r") as file:
        data = file.read().replace('\n', ' ')
        clean_text = []
        for line in data.split(" "):
            regex = re.compile(r"line_\d+.jpg\t")
            line = regex.sub("", line)
            clean_text.append(line)
        return clean_text

def find_true_values(image_path, data_dir = "/content/drive/MyDrive/TrainSpacy/OCR_testing/100.xlsx"):
    """
    Fetches the true values from the dataset for the given image
    """
    df = pd.read_excel(data_dir).dropna()
    true_values = []
    for index, row in df.iterrows():
        if row["image"] == os.path.basename(image_path):
            true_values.append(row["text"])
    return true_values
    # return true_values

def normalize_text(text):
    """
    This function normalizes the text by removing special characters and whitespaces
    """

    return re.sub(r'\W+', ' ', text).strip(".")

def test_accuracy_easy(image_path):
    """
    This function tests the accuracy of the Easy OCR model on the given image
    and returns the accuracy in percentage
    """

    easy_ocr_results = [normalize_text(result) for result in test_easy_ocr(f"{image_path}")]
    ocr_words = [word for result in easy_ocr_results for word in result.split()]
    true_values = [normalize_text(value) for value in find_true_values(f"{image_path}")]
    true_words = [word for value in true_values for word in value.split()]

    word_count= 0
    list_of_accs = []
    for true_word in true_words:
        for ocr_word in ocr_words:

            if ratio(true_word, ocr_word) > 0.9:
                word_count += 1

    acc = (word_count/len(true_words))*100
    list_of_accs.append(acc)
    return list_of_accs

def test_accuracy_linas(image_path, locr_results):
    """
    This function tests the accuracy of the Linas OCR model on the given image
    and returns the accuracy in percentage
    """
    print("----------------------------------------------------------------")
    print("Words found by the algorithm", locr_results)
    locr_words = [word.lower() for result in locr_results for word in result.split()]
    true_values = [normalize_text(value) for value in find_true_values(f"{image_path}")]
    true_words = [word.lower() for value in true_values for word in value.split()]
    print("The True values are", true_values)
    print("----------------------------------------------------------------")

    if len(true_words) == 0:
        print(f"No true words found for {os.path.basename(image_path)}. Cannot compute accuracy.")

        return [0]  # Returning a list with 0 to maintain the structure used in your plotting function
    word_count = 0
    list_of_accs = []
    for true_word in true_words:
        for ocr_word in locr_words:
            if ratio(true_word, ocr_word) > 0.9:
                print(f"True word: {true_word}, OCR word: {ocr_word}")
                word_count += 1

    acc = (word_count / len(true_words)) * 100 if len(true_words) > 0 else 0
    if acc == 0:
        with open("log.txt", "w") as file:
            file.write(f"This Image: {os.path.basename(image_path)} could not detect any accuracy.")

    list_of_accs.append(acc)
    return list_of_accs





def plot_accuracy_comparison(easy_ocr_accuracies, linas_ocr_accuracies, filenames, average_easy, average_linas):
    easy_ocr_accuracies = [acc for sublist in easy_ocr_accuracies for acc in sublist]
    linas_ocr_accuracies = [acc for sublist in linas_ocr_accuracies for acc in sublist]
    print("Easy OCR Accuracies", easy_ocr_accuracies)
    print("Linas OCR Accuracies", linas_ocr_accuracies)
    fig, ax = plt.subplots(figsize=(20, 8))

    index = np.arange(len(filenames))
    bar_width = 0.35
    # Plot bars with specific colors
    ax.bar(index, easy_ocr_accuracies, bar_width, label='Cropped White Images', color='skyblue')
    ax.bar(index + bar_width, linas_ocr_accuracies, bar_width, label='Not Cropped Images', color='lightgreen')

    # Plot average accuracy lines
    ax.axhline(y=average_easy, color='blue', linestyle='--', linewidth=2, label='Not Cropped OCR Accuracy')
    ax.axhline(y=average_linas, color='green', linestyle='--', linewidth=2, label='White Cropped OCR Accurac')

    ax.set_xlabel('Images')
    ax.set_ylabel('Accuracy (%)')
    ax.set_title('OCR Accuracy Comparison by Cropped and not Cropped')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(filenames, rotation=45, ha="right")
    ax.legend()
    plt.tight_layout()
    plt.savefig("accuracy_comparison_cropped_vs_not_cropped.png")
    plt.show()


if __name__ == "__main__":
    cropped_images_dir = "/content/drive/MyDrive/TrainSpacy/OCR_testing/Linas100_cropped"
    white_cropped = []
    easyocr_cropped = []
    filenames_list = []
    iteration_count = 0
    max_iterations = 95
    linas_path = "/content/drive/MyDrive/TrainSpacy/OCR_testing/Linas100"
    for filename in os.listdir(linas_path):
        if filename == ".DS_Store":
            continue

        if iteration_count >= max_iterations:
            break

        base_image_path = filename.rstrip('.jpg') + '.att.jpg'
        image_path = os.path.join(linas_path, filename)
        cropped_image_path = crop_image(image_path, cropped_images_dir)

        ## This code takes the original image as input
        # locr_cropped = ocr_module.LinasOCR(image_path)
        # locr_cropped.read_text()
        # locr_results = test_linas_ocr()
        # linas_ocr_accuracies = test_accuracy_linas(base_image_path, locr_results)



        ## Takes the cropped image as input
        # locr_cropped = ocr_module.LinasOCR(cropped_image_path)
        # locr_cropped.read_text()
        # locr_results = test_linas_ocr()

        # linas_ocr_accuracies = test_accuracy_linas(base_image_path, locr_results)
#
        easy_ocr_accuracies = test_accuracy_easy(cropped_image_path)
        easyocr_cropped.append(easy_ocr_accuracies)
        print("Easy OCR Accuracies", easyocr_cropped)


        ## This code executes on a white cropped image
        cropped_image_path2 = crop_image1(image_path, cropped_images_dir)
        print("Cropped Image Path 2", cropped_image_path2)
        locr_not_cropped = LinasOCR(cropped_image_path2)
        locr_not_cropped.read_text()
        locr_not_cropped = test_linas_ocr()
        ## Base image path is the one in the Dataset Linas100
        not_cropped_accuracy = test_accuracy_linas(base_image_path, locr_not_cropped)
        white_cropped.append(not_cropped_accuracy)


        filenames_list.append(filename)

        iteration_count += 1
        print("Iteration :" , iteration_count)

    average_linas = np.mean(white_cropped)
    average_easy = np.mean(easyocr_cropped)

    print("Average Linas Accuracy", average_linas)
    print("Average White Cropped OCR Accuracy", average_easy)

    plot_accuracy_comparison(white_cropped, easyocr_cropped, filenames_list, average_linas, average_easy)





image 1/1 /content/drive/MyDrive/TrainSpacy/OCR_testing/Linas100/sp69202819780870969418.jpg: 640x480 1 label1, 8.7ms
Speed: 4.2ms preprocess, 8.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict3
Easy OCR Accuracies [[20.0]]

image 1/1 /content/drive/MyDrive/TrainSpacy/OCR_testing/Linas100/sp69202819780870969418.jpg: 640x480 1 label1, 8.4ms
Speed: 4.2ms preprocess, 8.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)
Cropped Image Path 2 /content/drive/MyDrive/TrainSpacy/OCR_testing/Linas100_cropped/white_bg_sp69202819780870969418.jpg
Starting processing...
250 superpixels extracted; 7 line clusters generated
End of processing. Inference time: 46 seconds
----------------------------------------------------------------
Words found by the algorithm ['Plantae', 'groenlandicae', 'a', 'Museo', 'botanico', 'Hauniensl', 'distributae.', 'Channærerien', 'latifolsen', '(L', 'Scop.', 'det', 'CNi', 'Jørghed.', 'Nørlandet.', 

KeyboardInterrupt: 